In [1]:
# Essential imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# --- 1. Load the dataset ---
try:
    # Load the dataset
    df = pd.read_csv('Database.csv')
except FileNotFoundError:
    print("Error: 'Database.csv' not found. Please ensure the file is in the current directory.")
    exit()

In [3]:
# --- 2. Convert to Time Series and Create Time Features FIRST ---
print("Converting to time series and creating temporal features...")
df['Time'] = pd.to_datetime(df['Time'].str.replace('-T', ' ', regex=False))
df = df.set_index('Time').sort_index()

# Create comprehensive time-based features
def function_to_add_time_features(df):
    dataset_with_features = df.copy()
    
    # Basic time features
    dataset_with_features['hour'] = dataset_with_features.index.hour
    dataset_with_features['day_of_week'] = dataset_with_features.index.dayofweek
    dataset_with_features['day_of_month'] = dataset_with_features.index.day
    dataset_with_features['month'] = dataset_with_features.index.month
    dataset_with_features['weekend'] = (dataset_with_features.index.dayofweek >= 5).astype(int)
    
    # Cyclical encoding for temporal patterns
    dataset_with_features['hour_sin'] = np.sin(2 * np.pi * dataset_with_features['hour']/24)
    dataset_with_features['hour_cos'] = np.cos(2 * np.pi * dataset_with_features['hour']/24)
    dataset_with_features['month_sin'] = np.sin(2 * np.pi * dataset_with_features['month']/12)
    dataset_with_features['month_cos'] = np.cos(2 * np.pi * dataset_with_features['month']/12)
    
    # Time of day categories
    dataset_with_features['is_night'] = ((dataset_with_features['hour'] >= 0) & (dataset_with_features['hour'] <= 5)).astype(int)
    dataset_with_features['is_morning'] = ((dataset_with_features['hour'] >= 6) & (dataset_with_features['hour'] <= 11)).astype(int)
    dataset_with_features['is_afternoon'] = ((dataset_with_features['hour'] >= 12) & (dataset_with_features['hour'] <= 17)).astype(int)
    dataset_with_features['is_evening'] = ((dataset_with_features['hour'] >= 18) & (dataset_with_features['hour'] <= 23)).astype(int)
    
    return dataset_with_features

df = function_to_add_time_features(df)
print(f"Time features added. New shape: {df.shape}")

Converting to time series and creating temporal features...
Time features added. New shape: (38879, 25)


In [4]:
# --- 3. Outlier Detection & Removal using 3.5*IQR ---
def function_to_remove_outliers_using_interquartile_range(data, columns):
    clean_data = data.copy()
    initial_shape = clean_data.shape[0]
    
    for column_name in columns:
        if column_name in clean_data.columns and pd.api.types.is_numeric_dtype(clean_data[column_name]):
            Q1 = clean_data[column_name].quantile(0.25)
            Q3 = clean_data[column_name].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3.5 * IQR
            upper_bound = Q3 + 3.5 * IQR
            
            # Filter rows within the acceptable range
            clean_data = clean_data[(clean_data[column_name] >= lower_bound) & (clean_data[column_name] <= upper_bound)]
            
    rows_removed = initial_shape - clean_data.shape[0]
    return clean_data, rows_removed

# List of all numerical columns (including new time features)
num_cols = ['DHI', 'DNI', 'GHI', 'Wind_speed', 'Humidity', 'Temperature', 'PV_production', 'Wind_production', 'Electric_demand', 'hour', 'day_of_week', 'day_of_month', 'month']

# Perform outlier removal
df_clean, rows_removed = function_to_remove_outliers_using_interquartile_range(df, num_cols)

print(f"Original shape: {df.shape}")
print(f"After outlier removal: {df_clean.shape}")
print(f"Total rows removed by IQR: {rows_removed}")

Original shape: (38879, 25)
After outlier removal: (38879, 25)
Total rows removed by IQR: 0


In [5]:
# --- 4. Create Lag Features for Time Series Models ---
def create_lag_features(df, target_columns, lag_periods=[1, 2, 3, 6, 12, 24]):
    df_lagged = df.copy()
    
    for column_name in target_columns:
        for lag in lag_periods:
            df_lagged[f'{column_name}_lag_{lag}'] = df_lagged[column_name].shift(lag)
    
    # Rolling statistics for temporal patterns
    for column_name in target_columns:
        df_lagged[f'{column_name}_rolling_mean_6'] = df_lagged[column_name].rolling(window=6, min_periods=1).mean()
        df_lagged[f'{column_name}_rolling_std_6'] = df_lagged[column_name].rolling(window=6, min_periods=1).std()
        df_lagged[f'{column_name}_rolling_max_12'] = df_lagged[column_name].rolling(window=12, min_periods=1).max()
    
    return df_lagged.bfill()  # Backward fill for missing values

# Create lag features for targets and key features
target_columns = ['PV_production', 'Wind_production', 'GHI', 'Wind_speed', 'Temperature']
df_final = create_lag_features(df_clean, target_columns)

print(f"After lag features. Final shape: {df_final.shape}")

After lag features. Final shape: (38879, 70)


In [6]:
# --- 5. Save Processed Data ---
# Reset index to keep Time as a column
df_final = df_final.reset_index()

# Drop original unnecessary columns
df_final = df_final.drop(columns=['Unnamed: 0'], errors='ignore')

# Save the cleaned data with time series features
df_final.to_csv("preprocess_data_timeseries.csv", index=False)

print("\n" + "~"*60)
print("PROCESSING COMPLETED!!")
print("~"*60)
print("Time series conversion done.")
print("Temporal features created.")
print("Outliers removed using 3.5*IQR.")
print("Lag features generated for time series models.")
print(f"Final dataset saved with {df_final.shape[1]} features")
print(f"Date range: {df_final['Time'].min()} to {df_final['Time'].max()}")
print("\nNew features created:")
new_features = [column_name for column_name in df_final.columns if 'lag' in column_name or 'rolling' in column_name or column_name in ['hour', 'day_of_week', 'month', 'weekend']]
for feature in new_features:
    print(f"  - {feature}")


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
PROCESSING COMPLETED!!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Time series conversion done.
Temporal features created.
Outliers removed using 3.5*IQR.
Lag features generated for time series models.
Final dataset saved with 70 features
Date range: 2019-01-01 00:00:00 to 2019-05-15 23:50:00

New features created:
  - hour
  - day_of_week
  - month
  - weekend
  - PV_production_lag_1
  - PV_production_lag_2
  - PV_production_lag_3
  - PV_production_lag_6
  - PV_production_lag_12
  - PV_production_lag_24
  - Wind_production_lag_1
  - Wind_production_lag_2
  - Wind_production_lag_3
  - Wind_production_lag_6
  - Wind_production_lag_12
  - Wind_production_lag_24
  - GHI_lag_1
  - GHI_lag_2
  - GHI_lag_3
  - GHI_lag_6
  - GHI_lag_12
  - GHI_lag_24
  - Wind_speed_lag_1
  - Wind_speed_lag_2
  - Wind_speed_lag_3
  - Wind_speed_lag_6
  - Wind_speed_lag_12
  - Wind_speed_lag_24
  - Temperature_lag_1
  - Temperature_l